In [2]:
import pandas as pd
import numpy as np
import re

import time

from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Imputer
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression, LinearRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

import pandas_profiling

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

/Users/Andrew/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/Users/Andrew/anaconda3/lib/python3.6/site-packages/pandas_profiling/plot.py:15: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/Users/Andrew/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/Andrew/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/Andrew/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <mo

In [3]:
df = pd.read_csv('./datasets/clean_train_data.csv')
df2 = pd.read_csv('./datasets/clean_test_data.csv')

In [4]:
df.head()

,Unnamed: 0,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_11,FLAG_DOCUMENT_18,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
2,2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,...,1,0,0,0,0.000000,0.000000,0.000000,0.000000,1.000000,4.000000
3,3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
4,4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,...,0,0,0,0,0.002108,0.001803,0.002787,0.009299,0.546902,1.983769


In [5]:
df2.head()

,Unnamed: 0,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_11,FLAG_DOCUMENT_18,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
2,2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,...,1,0,0,0,0.000000,0.000000,0.000000,0.000000,1.000000,4.000000
3,3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
4,4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,...,0,0,0,0,0.002108,0.001803,0.002787,0.009299,0.546902,1.983769


In [6]:
df.columns

Index(['Unnamed: 0', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE',
       'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'FLAG_MOBIL', 'FLAG_EMP_PHONE',
       'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL',
       'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
       'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY',
       'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY',
       'ORGANIZATION_TYPE', 'EXT_SOURCE_3', 'OBS_30_CNT_SOCIAL_CIRCLE',
       'DEF_30_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE',
       'DAYS_LAST_PHON

In [10]:
# Use pandas profiling to find some correlations
pandas_profiling.ProfileReport(df)

Number of variables,58
Number of observations,48744
Total Missing (%),0.0%
Total size in memory,21.6 MiB
Average record size in memory,464.0 B
Numeric,24
Categorical,12
Boolean,21
Date,0
Text (Unique),0
Rejected,1


In [7]:
df.corr()

,Unnamed: 0,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_11,FLAG_DOCUMENT_18,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
Unnamed: 0,1.000000,0.999972,0.000673,0.001283,0.005047,0.007124,0.003329,0.002328,-0.000870,0.001069,...,0.003072,0.003895,0.003023,-0.006293,-0.000371,0.000995,0.001079,0.000417,-0.001942,0.003225
SK_ID_CURR,0.999972,1.000000,0.000635,0.001278,0.005014,0.007110,0.003324,0.002325,-0.000845,0.001032,...,0.003071,0.003946,0.003054,-0.006286,-0.000287,0.001013,0.001102,0.000402,-0.001956,0.003233
CNT_CHILDREN,0.000673,0.000635,1.000000,0.038962,0.027840,0.056759,-0.015231,0.317877,-0.238319,0.175054,...,0.050427,-0.003874,-0.006461,-0.000862,0.005964,0.001443,0.007053,-0.007816,0.027192,-0.036809
AMT_INCOME_TOTAL,0.001283,0.001278,0.038962,1.000000,0.396572,0.457733,0.199773,0.054400,-0.154619,0.067973,...,0.173659,0.049128,-0.003945,-0.006624,0.009626,0.004696,-0.002698,0.008180,0.006974,0.003088
AMT_CREDIT,0.005047,0.005014,0.027840,0.396572,1.000000,0.777481,0.135694,-0.046169,-0.083483,0.030740,...,0.072205,0.025121,0.000614,-0.000197,-0.001035,0.004626,0.002752,-0.000148,-0.007344,-0.032722
AMT_ANNUITY,0.007124,0.007110,0.056759,0.457733,0.777481,1.000000,0.150783,0.047847,-0.137747,0.064439,...,0.130895,0.036684,-0.010615,-0.010762,0.007903,0.006265,0.002893,0.005340,0.011664,-0.042092
REGION_POPULATION_RELATIVE,0.003329,0.003324,-0.015231,0.199773,0.135694,0.150783,1.000000,-0.033339,0.000209,-0.053130,...,0.095708,0.041929,0.010878,0.013449,-0.001308,-0.010940,-0.007732,0.000097,-0.024764,0.000944
DAYS_BIRTH,0.002328,0.002325,0.317877,0.054400,-0.046169,0.047847,-0.033339,1.000000,-0.637129,0.320174,...,0.124355,0.018868,0.033619,0.030752,0.008203,-0.000357,0.011499,0.013047,0.082159,-0.088453
DAYS_EMPLOYED,-0.000870,-0.000845,-0.238319,-0.154619,-0.083483,-0.137747,0.000209,-0.637129,1.000000,-0.209123,...,-0.126805,-0.028651,-0.016180,-0.018859,-0.012179,-0.000736,-0.013179,-0.013023,-0.041579,0.060654
DAYS_REGISTRATION,0.001069,0.001032,0.175054,0.067973,0.030740,0.064439,-0.053130,0.320174,-0.209123,1.000000,...,0.058618,0.010140,0.009631,0.008590,0.005812,-0.000142,0.008133,0.006937,0.043049,-0.034512


In [10]:
df.NAME_CONTRACT_TYPE.unique()

array(['Cash loans', 'Revolving loans'], dtype=object)